In [1]:
from interpreting_neurons_utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


- Focus Games
    - Iterate over layer, neurons, games
    - Iterate over games
    - If neuron activation above threshold, do 
        - Get all probe features
        - Do Cosine Similiarity / logit attribution
        - Save Group of important Features (First just plot Histogram though)
        - print fraction of Groups

In [15]:
class ProbeMetadata:
    def __init__(self, name, direction_int, row, col):
        self.name = name
        self.direction_int = direction_int
        self.row = row
        self.col = col
        self.direction_str = get_direction_str(self.name, direction_int)
        self.t = (row, col)
        self.label = tuple_to_label(self.t)
        self.weights = None
        self.resid_dot_product = None

    def get_probe(self, layer, module="mid"):
        if self.weights is None:
            self.weights = get_probe(layer, self.name, module)[0, :, self.row, self.col, self.direction_int]
        return self.weights

    

In [16]:
import pickle

In [17]:
layer = 1
coefficients = pickle.load(open(f"coefficients/coefficients_L{layer}_{0}_{7}.pth", "rb"))
base_resid = pickle.load(open(f"coefficients/base_resid_{0}_{7}.pth", "rb"))

In [19]:
layer = 1
neuron = 426

module = f"blocks.{layer}.ln2.hook_normalized"

threshold = 0.0025

def get_relevant_probes(layer, game_idx, pos, neuron, everything = False):
    relevant_probes = []
    resid_mid = focus_cache[module][game_idx, pos]
    for probe_name in ["linear", "flipped", "placed"]:
        probe = get_probe(layer, probe_name, "mid")[0]
        if everything:
            probe_result = einops.einsum(resid_mid, probe, "d_model, d_model rows cols options -> rows cols options").softmax(dim=-1)
            for row in range(8):
                for col in range(8):
                    for direction_int in range(probe_result.shape[2]):
                        # direction_int = probe_result[row, col].item()
                        probe_meta = ProbeMetadata(probe_name, direction_int, row, col)
                        probe_meta.resid_dot_product = probe_result[row, col, direction_int].item()
                        relevant_probes.append(probe_meta)
        else:
            probe_result = einops.einsum(resid_mid, probe, "d_model, d_model rows cols options -> rows cols options").argmax(dim=-1)
            for row in range(8):
                for col in range(8):
                    direction_int = probe_result[row, col].item()
                    if direction_int == 1 and (probe_name == "flipped" or probe_name == "placed"):
                        continue
                    probe_meta = ProbeMetadata(probe_name, direction_int, row, col)
                    relevant_probes.append(probe_meta)
    return relevant_probes

def get_attribution_score(layer, neuron, probe_meta, everything = False):
    neuron_weighs = get_w_in(model, layer, neuron)
    if everything:
        probe : Float[Tensor, "d_model"] = probe_meta.get_probe(layer) * probe_meta.resid_dot_product
    else:
        probe : Float[Tensor, "d_model"] = probe_meta.get_probe(layer) * coefficients[probe_meta.name][probe_meta.direction_str][probe_meta.t]
    return einops.einsum(neuron_weighs, probe, "d_model, d_model -> ").item()

count = 0

for game_idx in range(50):
    for pos in range(59):
        neuron_act = focus_cache["mlp_post", layer][game_idx, pos, neuron]
        if neuron_act < threshold:
            continue
        print(f"Game {game_idx}, pos {pos}, neuron {neuron}, neuron_act {neuron_act}")
        relevant_probes = get_relevant_probes(layer, game_idx, pos, neuron)
        for probe_meta in relevant_probes:
            probe_meta.attribution_score = get_attribution_score(layer, neuron, probe_meta)
        relevant_probes.sort(key=lambda x: x.attribution_score, reverse=True)
        scores = [probe_meta.attribution_score for probe_meta in relevant_probes]
        # plot histogram of scores using plotly px
        # plotly histogram
        lables = [f"{probe_meta.label} {probe_meta.direction_str}" for probe_meta in relevant_probes]
        # fig = px.histogram(scores, nbins=50)
        fig = px.scatter(y=lables, x=scores)
        fig.update_layout(height = 3000)
        fig.show()
        count += 1
        if count >= 1:
            break
    if count >= 1:
            break



Game 0, pos 47, neuron 426, neuron_act 0.6182472705841064
kock


In [ ]:
neurons = [t.Tensor([neuron]).to(device).to(t.int64)]

probe_directions_new = {
    "linear" : ["empty", "mine", "yours"],
    "flipped" : ["flipped", "not_flipped"],
    "placed" : ["placed"],
}

plot_neuron_weights(neurons, layer, title=f"Neuon Weights L{layer}_N{neuron} Projected to different Linear Probes", probe_names_and_directions=probe_directions_new)

In [ ]:
# top_games, top_moves = get_max_acitvations_of_neuron(focus_cache, layer, neuron, 10)
# for i in range(len(top_games)):
game = 0
pos = 47
# pos = top_moves[i].item()
vis_args = VisualzeBoardArguments()
vis_args.start_pos = pos-5
vis_args.end_pos = pos+1
# vis_args.include_attn_only = True
# vis_args.include_mlp_only = True
# plot_game(focus_games_string, game)

visualize_game(
    focus_games_string[game, :59],
    vis_args,
    model,
)
# break

torch.Size([8, 1, 59, 8, 8])
